In [1]:
import csv
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='H_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for pr in pr_list:
                pr_arr_row.append(row[pr])

            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float64)
    y = np.asarray(y_arr, dtype=np.float64)
    return X, y

#### Сумма, средний, высший, низший уровни

In [3]:
def get_sum(h_max):
    return np.sum(h_max)
    
def get_avg(h_max):
    return np.mean(h_max)
    
def get_max(h_max):
    return np.amax(h_max)
    
def get_min(h_max):
    return np.amin(h_max)

#### Среднеквадратическая погрешность прогноза S

In [4]:
def get_s(h_max, h_forecast=None):
    # Среднеквадратическая погрешность прогноза
    n = h_max.shape[0]
    sqr_diff = np.sum((h_max - h_forecast) ** 2) / (n - 1)
    std = sqr_diff ** 0.5
    return std    

#### Среднеквадратическое отклонение sigma

In [5]:
def get_sigma(h_max):
    # Среднеквадратическая погрешность климатическая.
    # Рассчитывается только по всей совокупности данных.
    return np.std(h_max, ddof=1)

In [6]:
def get_hmax_avg(h_max):
    # Среднее значение h_max.
    # Рассчитывается только по всей совокупности данных.
    return np.mean(h_max)

#### Допустимая погрешность прогноза delta_dop

In [7]:
def get_delta_dop(sigma):
    return 0.674 * sigma

#### Критерий эффективности метода прогнозирования климатический S/sigma

In [8]:
def get_criterion(s, sigma):
    return s / sigma

#### Климатическая обеспеченность Pk

In [9]:
def get_pk(h_max, h_max_avg, delta_dop):
    #avg_level = np.mean(h_max)
    diff = np.abs(h_max - h_max_avg) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Обеспеченность метода (оправдываемость) Pm

In [10]:
def get_pm(h_max, h_forecast, delta_dop):
    diff = np.abs(h_max - h_forecast) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Корреляционное отношение

In [11]:
def get_correlation_ratio(criterion):
    c_1 = (1 - criterion ** 2)
    ro = c_1 ** 0.5 if c_1 > 0 else 0
    return ro

#### Вероятная ошибка прогноза S'

In [12]:
def get_forecast_error(s):
    return 0.674 * s

#### Ошибки климатического/природного прогноза для каждого года delta50

In [13]:
def get_delta50(h_max, delta_dop, h_max_avg=None, h_max_forecast=None):
    if h_max_forecast is None:
        # delta50 климатическая
        return (h_max - h_max_avg) / delta_dop
    else:
        # delta50 прогноза
        return (h_max - h_max_forecast) / delta_dop
  

#### Функция записи в csv файл

In [14]:
import csv
def write_dataset_csv(dataset, dataset_name, fieldnames, pr_group):
    with open(f'results/{dataset_name}/group-{pr_group}/{dataset_name}-гр{pr_group}.csv', 'w', newline='', encoding='utf-8') as csvfile:# , encoding='utf-8'
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


#### Функция разделения набора данных на тренировочный и тестовый

In [15]:
def train_test_split(X, y, n_test):
    X_train = X[:-n_test]
    y_train = y[:-n_test]
    X_test = X[-n_test:]
    y_test = y[-n_test:]
    return X_train, y_train, X_test, y_test

#### Функция формирования тестового набора данных с подстановкой нормированных значений

In [16]:
def test_norm(x, pr_list, norms):
    x_norm = np.copy(x)
    for col, pr in enumerate(pr_list):
        if pr in norms:
            x_norm[:, col:col+1] = norms[pr]
    return x_norm

#### Функция получения датасетов

In [17]:
def get_datasets():
    datasets = {
        # 'Неман-Белица': 'Неман',
        # 'Неман-Гродно': 'Неман',
        'Неман-Мосты': 'Неман',
        'Неман-Столбцы': 'Неман',
    }
    return datasets

#### Функция получения списка предикторов по названию датасета

In [18]:
def get_predictors(dataset_name, pr_group):

    datasets = get_datasets()   
    
    # predictors_lists = {
    #     'Неман': ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs'],
    # }

    predictors_lists = {
        'Неман': (
            ['S_2802', 'Smax', 'H', 'X', 'X1', 'X2', 'X3', 'X4', 'Xs'],
            ['Smax', 'H', 'X', 'X1', 'X3'],
            ['S_2802', 'H', 'X2', 'X3', 'Xs'],
        )
    }
    return predictors_lists[datasets[dataset_name]][pr_group]
    

In [19]:
def get_norms(dataset_name):
    norms_list = {
        # 'Неман-Белица': {'X1': 46.0, 'X2':35.0},
        # 'Неман-Гродно': {'X1': 36.0, 'X2':26.0},
        'Неман-Мосты': {'x1': 40.0, 'x2':31.0},
        'Неман-Столбцы': {'x1': 43.0, 'x2':34.0},
    }
    return norms_list[dataset_name]

    

In [20]:
np.logspace(-2, 3, num=100)

array([1.00000000e-02, 1.12332403e-02, 1.26185688e-02, 1.41747416e-02,
       1.59228279e-02, 1.78864953e-02, 2.00923300e-02, 2.25701972e-02,
       2.53536449e-02, 2.84803587e-02, 3.19926714e-02, 3.59381366e-02,
       4.03701726e-02, 4.53487851e-02, 5.09413801e-02, 5.72236766e-02,
       6.42807312e-02, 7.22080902e-02, 8.11130831e-02, 9.11162756e-02,
       1.02353102e-01, 1.14975700e-01, 1.29154967e-01, 1.45082878e-01,
       1.62975083e-01, 1.83073828e-01, 2.05651231e-01, 2.31012970e-01,
       2.59502421e-01, 2.91505306e-01, 3.27454916e-01, 3.67837977e-01,
       4.13201240e-01, 4.64158883e-01, 5.21400829e-01, 5.85702082e-01,
       6.57933225e-01, 7.39072203e-01, 8.30217568e-01, 9.32603347e-01,
       1.04761575e+00, 1.17681195e+00, 1.32194115e+00, 1.48496826e+00,
       1.66810054e+00, 1.87381742e+00, 2.10490414e+00, 2.36448941e+00,
       2.65608778e+00, 2.98364724e+00, 3.35160265e+00, 3.76493581e+00,
       4.22924287e+00, 4.75081016e+00, 5.33669923e+00, 5.99484250e+00,
      

In [21]:
# from sklearn.utils.fixes import loguniform
# loguniform(1e-2, 1e0)

# import scipy.stats as stats
# stats.uniform(0, 1)

# np.logspace(-7, -5, num=10)

#np.linspace(0.01, 1.0, num=50)

#np.arange(0.1, 1.5, 50)
np.arange(1, 11)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

#### Функция обучения и оценки моделей

In [22]:
def compare_models(pr_group, n_test=None, norms=True, top_best=None):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge, RidgeCV
    from sklearn.linear_model import Lasso, LassoCV
    from sklearn.linear_model import ElasticNet, ElasticNetCV
    from sklearn.linear_model import Lars, LarsCV
    from sklearn.linear_model import LassoLars
    from sklearn.linear_model import OrthogonalMatchingPursuit
    from sklearn.linear_model import BayesianRidge
    from sklearn.linear_model import ARDRegression
    from sklearn.linear_model import TweedieRegressor
    from sklearn.linear_model import SGDRegressor
    from sklearn.linear_model import PassiveAggressiveRegressor
    from sklearn.linear_model import HuberRegressor
    from sklearn.linear_model import TheilSenRegressor
    from sklearn.linear_model import QuantileRegressor

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF

    import scipy.stats as stats
    from sklearn.pipeline import Pipeline, make_pipeline
    from sklearn.feature_selection import SelectKBest, SelectFromModel
    from sklearn.feature_selection import r_regression

    
    
    
    ds_dir = 'data' # В константы
    
    names = [
        'LinearRegression',
        'Ridge_GridSearchCV',
        'Lasso_GridSearchCV',
        'ElasticNet_GridSearchCV',
        'Lars1',
        'Lars2',
        'Lars3',
        'Lars4',
        'Lars5',
        # 'Lars6',
        # 'Lars7',
        # 'Lars8',
        'LassoLars_GridSearchCV',
        'OMP1',
        'OMP2',
        'OMP3',
        'OMP4',
        'OMP5',
        'BayesianRidge',
        'ARDRegression',
        'TweedieRegressor_GridSearchCV',
        'SGDRegressor', 
        'PassiveAggressiveRegressor',
        'HuberRegressor',
        'TheilSenRegressor',
        'QuantileRegressor',
        
        
        
        #'Perceptron'
        #'GaussianProcessRegressor',
        
    ]
    rng = np.random.RandomState(0)
    alphas = np.logspace(-2, 3, num=100)
    sgd_alphas = np.logspace(-4, 1, num=100)
    l1_ratio = np.linspace(0.01, 1.0, num=50)

    alphas_lambdas = np.logspace(-7, -5, num=16)
    threshold_lambdas = np.arange(5000, 15000, step=10)
    powers = np.arange(0, 4)
    losses = ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
    cc = np.linspace(0.1, 1.5, 50)
    epsilons = np.linspace(1, 2, 10)
    huber_alphas = np.logspace(-4, -1, num=20)
    quantiles = [0.25, 0.5, 0.75]
    q_alphas = np.linspace(0.25, 2, 20)
    
    
    
    # n_nonzero_coefs = np.arange(1, len(get_predictors(ds, pr_group)))
    
    regressors = [
        LinearRegression(),
        GridSearchCV(estimator=Ridge(random_state=rng), param_grid={"alpha": alphas}),
        GridSearchCV(estimator=Lasso(random_state=rng), param_grid={"alpha": alphas}),
        GridSearchCV(estimator=ElasticNet(random_state=rng), param_grid={"alpha": alphas, "l1_ratio": l1_ratio}, n_jobs=-1),
        Lars(n_nonzero_coefs=1),
        Lars(n_nonzero_coefs=2),
        Lars(n_nonzero_coefs=3),
        Lars(n_nonzero_coefs=4),
        Lars(n_nonzero_coefs=5),
        # Lars(n_nonzero_coefs=6),
        # Lars(n_nonzero_coefs=7),
        # Lars(n_nonzero_coefs=8),
        GridSearchCV(estimator=LassoLars(random_state=rng), param_grid={"alpha": alphas}),
        OrthogonalMatchingPursuit(n_nonzero_coefs=1),
        OrthogonalMatchingPursuit(n_nonzero_coefs=2),
        OrthogonalMatchingPursuit(n_nonzero_coefs=3),
        OrthogonalMatchingPursuit(n_nonzero_coefs=4),
        OrthogonalMatchingPursuit(n_nonzero_coefs=5),
        BayesianRidge(tol=1e-6),
        ARDRegression(),
        GridSearchCV(estimator=TweedieRegressor(), param_grid={"power": powers, "alpha": alphas}, n_jobs=-1),
        GridSearchCV(estimator=SGDRegressor(random_state=rng), param_grid={"loss": losses, "alpha": sgd_alphas}, n_jobs=-1),
        GridSearchCV(estimator=PassiveAggressiveRegressor(random_state=rng), param_grid={"C": cc}, n_jobs=-1, cv=3),
        HuberRegressor(),
        TheilSenRegressor(),
        GridSearchCV(estimator=QuantileRegressor(), param_grid={"alpha": q_alphas, "quantile": quantiles}, n_jobs=-1),
        
        #"quantile": quantiles,

        ##PassiveAggressiveRegressor(random_state=rng),
        ##GridSearchCV(estimator=HuberRegressor(), param_grid={"epsilon": epsilons, "alpha": huber_alphas}, n_jobs=-1),
        ##GridSearchCV(estimator=BayesianRidge(tol=1e-6), param_grid={"alpha_1": alphas_lambdas, "alpha_2": alphas_lambdas, "lambda_1": alphas_lambdas, "lambda_2": alphas_lambdas}, n_jobs=-1),       
        #GaussianProcessRegressor(kernel=RBF(length_scale=1.1) + WhiteKernel() + DotProduct(), random_state=0)
        #GridSearchCV(Pipeline(steps=[('feature_selection', SelectKBest(r_regression, k=4)), ('regression', Lasso(random_state=rng))]), param_grid={"regression__alpha": alphas}),
        #GridSearchCV(Pipeline(steps=[('feature_selection', SelectFromModel(Lars(n_nonzero_coefs=4))), ('regression', Lasso(random_state=rng))]), param_grid={"regression__alpha": alphas}),
    ]

    datasets = get_datasets()

    fieldnames = ['Predictors', 'Equations', 'Method', 'Criterion', 'Correlation', 'Pm']

    # datasets_result = {
    #     "hydropost_0": [
    #         { model_row }
    #         { model_row }
    #     ],
    #     ...,
    #     "hydropost_n": [
    #         { model_row }
    #         { model_row }
    #     ],
    # }
    
    
    # Итерация по датасетам
    datasets_result = dict()
    for ds in datasets:
        result_list = []
        
        pr_list = get_predictors(ds, pr_group)
        
        X, y = get_river_dataset(f'{ds_dir}/{ds}.csv', pr_list=pr_list)

        if n_test is not None and n_test != 0:
            X_train, y_train, X_test, y_test = train_test_split(X, y, n_test)
        else:
            X_train = X[:]
            y_train = y[:]
            X_test = X_train
            y_test = y_train

        if norms:
            norms = get_norms(ds)
            X_test = test_norm(X_test, pr_list, norms)
            
        # Итерация по моделям регрессии
        for name, regr in zip(names, regressors):
            one_model_row = dict()

            # try:
            regr.fit(X_train, y_train)
            y_predicted = regr.predict(X_test)
            # except Exception:
                # continue
            
            # pipe_params = regr.get_params()
            # print(pipe_params)

            try:
                coef = regr.best_estimator_.coef_
                intercept = regr.best_estimator_.intercept_
                
                if isinstance(intercept, np.ndarray):
                    intercept = intercept[0]
                
                # coef = regr.best_estimator_.named_steps['regression'].coef_
                # intercept = regr.best_estimator_.named_steps['regression'].intercept_
                # print('cv_coef', coef)
                print('cv_intercept', intercept, type(intercept))
                # print(ds, regr.best_estimator_.alpha_)
            except Exception as error:
                coef = regr.coef_
                intercept = regr.intercept_
                
                if isinstance(intercept, np.ndarray):
                    intercept = intercept[0]
                
                # print('rg_coef', coef)
                print('rg_intercept', intercept, type(intercept))
                # print(ds, regr.alpha_)
                print(error)
                
            # Коэффициенты уравнения (если есть)
            print('COEF', coef)
            coef = np.around(coef, 2)
            intercept = round(intercept, 2)
            try:
                predictors_coef = {f: c for f, c in zip(pr_list, coef) if c != 0.0}
                predictors = ", ".join(predictors_coef.keys())
                #predictors = predictors.upper()
                print(intercept, predictors_coef.items())
                #equation = str(round(intercept, 2)) + ' ' + ' '.join(str(round(c, 2))+'*'+f for f, c in predictors_coef.items())
                equation = str(intercept) + ' ' + ' '.join(str(c) + '*' + f for f, c in predictors_coef.items())
                equation = equation.replace(" -", "-")
                equation = equation.replace(" ", " + ")
                equation = equation.replace("-", " - ")
                #equation = equation.upper()
    
                one_model_row['Predictors'] = predictors
                one_model_row['Equations'] = equation
            except Exception as error:
                print(error)
                one_model_row['Predictors'] = ""
                one_model_row['Equations'] = ""

            # Название датасета
            one_model_row['Dataset_name'] = ds

            # Группа предикторов
            one_model_row['Group'] = pr_group
                
            # Название метода
            one_model_row['Method'] = name

            # Расчет показателей качества по методике
            
            one_model_row['H_sum'] = get_sum(y)
            one_model_row['H_max'] = get_max(y)
            one_model_row['H_min'] = get_min(y)
            
            # Среднее значение максимального уровня по всей выборке
            h_max_avg = get_hmax_avg(y)
            one_model_row['H_avg'] = h_max_avg
            
            # Среднеквадратическое отклонение
            sigma = get_sigma(y)
            one_model_row['Sigma'] = sigma

            # Допустимая погрешность прогноза
            delta_dop = get_delta_dop(sigma)
            one_model_row['Delta_dop'] = delta_dop

            # Обеспеченность климатическая Pk
            pk = get_pk(y_test, h_max_avg, delta_dop)
            one_model_row['Pk'] = round(pk, 8)

            # Обеспеченность метода (оправдываемость) Pm
            pm = get_pm(y_test, y_predicted, delta_dop)
            one_model_row['Pm'] = round(pm, 8)

            # Среднеквадратическая погрешность прогноза
            s_forecast = get_s(y_test, y_predicted)
            one_model_row['S'] = s_forecast
            
            # Критерий эффективности метода прогнозирования климатический S/sigma
            criterion_forecast = get_criterion(s_forecast, sigma)
            one_model_row['Criterion'] = criterion_forecast

            # Критерий эффективности метода прогнозирования климатический S/sigma в квадрате
            criterion_sqr = get_criterion(s_forecast, sigma) ** 2.0
            one_model_row['Criterion_sqr'] = criterion_sqr
            
            # Корреляционное отношение ro
            correlation_forecast = get_correlation_ratio(criterion_forecast)
            one_model_row['Correlation'] = correlation_forecast
                        
            # Model
            one_model_row['Model'] = regr

            # models_list.append(one_model_row)
            result_list.append(one_model_row)

        # Сортировка результатов по каждому датасету
        result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))

        datasets_result[ds] = result_list

        # Запись в .csv файл
        write_dataset_csv(result_list, ds, fieldnames, pr_group=pr_group)

        for i, rl in enumerate(result_list):
            if top_best is not None:
                if i >= top_best:
                    break
            verify_forecast(ds, rl, i, pr_group=pr_group, n_test=n_test)

    return datasets_result

#### Функция формирования проверочных прогнозов

In [23]:
def verify_forecast(dataset_name, rl, num, pr_group, n_test=None, norms=True):

    ds_dir = 'data' # В константы

    pr_list = get_predictors(dataset_name, pr_group)
    pr_list = ['year'] + pr_list
    
    fieldnames = [
        '№', 'Год',
        'Hmax фактический', 'Hф-Hср', '(Hф-Hср)^2', 'Погрешность климатических прогнозов в долях от допустимой погрешности',
        'Hmax прогнозный', 'Hф-Hп', '(Hф-Hп)^2', 'Погрешность проверочных прогнозов в долях от допустимой погрешности',
    ]

    # fieldnames = [
    #     'Year',
    #     'Hmax fact', 'Hf-Havg', '(Hf-Havg)^2', 'Error climate',
    #     'Hmax forecast', 'Hf-Hfor', '(Hf-Hfor)^2', 'Error forecast',
    # ]

    X, y = get_river_dataset(f'{ds_dir}/{dataset_name}.csv', pr_list=pr_list, y_name='H_max')

    if n_test is not None and n_test != 0:
        _, _, X_test, y_test = train_test_split(X, y, n_test)
    else:
        X_test = X
        y_test = y

    if norms:
        norms = get_norms(dataset_name)
        X_test = test_norm(X_test, pr_list, norms)

    # Выделение первой колонки (года) из набора предикторов
    years = X_test[:, 0]
    X_test = X_test[:, 1:]
    
    # Forecast
    #h_max_forecast = np.around(model.predict(X_test))
    h_max_forecast = rl['Model'].predict(X_test)
    
    # Hсредний
    #h_max_avg = np.around(np.mean(y))
    h_max_avg = np.mean(y)

    # H - Hсредний
    diff_fact = y_test - h_max_avg

    # (H - Hсредний) в квадрате
    diff_fact_sqr = diff_fact ** 2

    # Погрешность климатических прогнозов в долях от допустимой погрешности
    delta_dop = get_delta_dop(get_sigma(y))
    error_climate = np.around(get_delta50(y_test, delta_dop, h_max_avg=h_max_avg), decimals=2)

    # H - Hпрогнозный
    diff_forecast = y_test - h_max_forecast

    # (H - Hпрогнозный) в квадрате
    diff_forecast_sqr = diff_forecast ** 2       

    # Погрешность проверочных прогнозов в долях от допустимой погрешности
    error_forecast = np.around(get_delta50(y_test, delta_dop, h_max_forecast=h_max_forecast), decimals=2)

    # Номер по порядку
    rows_num = y_test.shape[0]
    npp = np.arange(1, rows_num + 1, 1)

    # Конкатенация массивов
    att_tuple = (npp, years, y_test, diff_fact, diff_fact_sqr, error_climate, h_max_forecast, diff_forecast, diff_forecast_sqr, error_forecast)
    arr = np.column_stack(att_tuple)
    arr = arr.tolist()

    # Обеспеченность метода (оправдываемость) Pm
    pm = get_pm(y_test, h_max_forecast, delta_dop)
    
    # Запись проверочного прогноза в csv файл
    with open(f'results/{dataset_name}/group-{pr_group}/{dataset_name}-проверочный-гр{pr_group}-{num:0>2}.csv', 'w', newline='', encoding='utf-8') as csvfile: #, encoding='utf-8'
        stat_str = (
            f"Таблица  - Проверочные прогнозы максимумов весеннего\n"
            f"половодья р.{rl['Dataset_name']} по данным, рассчитанным\n"
            f"на дату 28 февраля\n"
            f"Предикторы;; {rl['Predictors']}\n"
            f"Уравнение;; {rl['Equations']}\n\n"
        )
        csvfile.write(stat_str)
       
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(fieldnames)
        writer.writerows(arr)
        
      
        
        

In [24]:
top_best = 3
result = compare_models(pr_group=2, n_test=0, norms=True, top_best=top_best)
for key in result:
    print(key)
    for num, r in enumerate(result[key]):
        dataset_name = r['Dataset_name']
        pr_group = r['Group']
        stat_str = (
            f"Сумма;;{r['H_sum']}\n"  
            f"Средний;;{r['H_avg']}\n" 
            f"Высший;;{r['H_max']}\n"
            f"Низший;;{r['H_min']}\n\n"
            
            f"σ = ;;{r['Sigma']};;σ -;среднеквадратическое отклонение (см)\n" 
            f"δдоп =;;{r['Delta_dop']};;δдоп -;допустимая погрешность прогноза (см)\n" 
            f"Pк =;;{r['Pk']};;Pк -;климатическая обеспеченность в %\n"
            f"Pм =;;{r['Pm']};;Pм -;обеспеченность метода (допустимой погрешности проверочных прогнозов) в %\n"
            f"S =;;{r['S']};;S -;среднеквадратическая погрешность (см)\n"
            f"S/σ =;;{r['Criterion']};;S/σ -;критерий эффективности метода прогнозирования\n" 
            f"(S/σ)^2 =;;{r['Criterion_sqr']};;ρ -;корреляционное отношение\n"
            f"ρ =;;{r['Correlation']};;;(оценка эффективности метода прогнозирования)\n"
            f"Модель:;;{r['Method']};;δ50% -;погрешность (ошибка) прогнозов (см)\n"
            
            
            # f"Sigma;{r['Sigma']},\n" 
            # f"Delta_dop;{r['Delta_dop']},\n" 
            # f"Pk;{r['Pk']},\n"
            # f"Pm;{r['Pm']},\n"
            # f"S;{r['S']},\n"
            # f"Criterion;{r['Criterion']},\n" 
            # f"Criterion^2;{r['Criterion_sqr']},\n"
            # f"Correlation;{r['Correlation']},\n"            
            # f"Модель:; {r['Method']}\n"
        )

        

        with open(f'results/{dataset_name}/group-{pr_group}/{dataset_name}-проверочный-гр{pr_group}-{num:0>2}.csv', 'a', newline='', encoding='utf-8') as csvfile: # , encoding='utf-8'
            csvfile.write(stat_str)

        
        #print(stat_str)
            


rg_intercept -14.215147661345327 <class 'numpy.float64'>
'LinearRegression' object has no attribute 'best_estimator_'
COEF [1.53739981e+00 1.02694297e+00 1.44918207e+00 1.11350631e-03
 3.57720984e-01]
-14.22 dict_items([('S_2802', 1.54), ('H', 1.03), ('X2', 1.45), ('Xs', 0.36)])
cv_intercept -6.976709338522568 <class 'numpy.float64'>
COEF [1.51580117 0.99257326 1.35674303 0.00896744 0.35295806]
-6.98 dict_items([('S_2802', 1.52), ('H', 0.99), ('X2', 1.36), ('X3', 0.01), ('Xs', 0.35)])
cv_intercept 105.24153560428343 <class 'numpy.float64'>
COEF [1.3895811  0.64418698 0.37778069 0.         0.        ]
105.24 dict_items([('S_2802', 1.39), ('H', 0.64), ('X2', 0.38)])
cv_intercept 55.552861798526294 <class 'numpy.float64'>
COEF [1.2145014  0.7177985  0.74688199 0.05187379 0.30046336]
55.55 dict_items([('S_2802', 1.21), ('H', 0.72), ('X2', 0.75), ('X3', 0.05), ('Xs', 0.3)])
rg_intercept 213.2614428039086 <class 'numpy.float64'>
'Lars' object has no attribute 'best_estimator_'
COEF [0.88146 

C:\projects\pkogo\env_ogo\lib\site-packages\sklearn\linear_model\_quantile.py:186: FutureWarning: The default solver will change from 'interior-point' to 'highs' in version 1.4. Set `solver='highs'` or to the desired solver to silence this warning.
  warnings.warn(


cv_intercept 50.34742051219028 <class 'numpy.float64'>
COEF [ 1.53413761e+00  5.50851661e-01  7.52878559e-01 -2.83169174e-12
  1.52208817e-01]
50.35 dict_items([('S_2802', 1.53), ('H', 0.55), ('X2', 0.75), ('Xs', 0.15)])
rg_intercept 83.41642923484402 <class 'numpy.float64'>
'LinearRegression' object has no attribute 'best_estimator_'
COEF [0.51782745 0.28057993 0.77068663 0.19228919 0.15058787]
83.42 dict_items([('S_2802', 0.52), ('H', 0.28), ('X2', 0.77), ('X3', 0.19), ('Xs', 0.15)])
cv_intercept 87.16936788360829 <class 'numpy.float64'>
COEF [0.51091741 0.26413491 0.73086703 0.19115997 0.14655702]
87.17 dict_items([('S_2802', 0.51), ('H', 0.26), ('X2', 0.73), ('X3', 0.19), ('Xs', 0.15)])
cv_intercept 103.29946588048414 <class 'numpy.float64'>
COEF [0.49227484 0.19795876 0.64170766 0.17376187 0.1136011 ]
103.3 dict_items([('S_2802', 0.49), ('H', 0.2), ('X2', 0.64), ('X3', 0.17), ('Xs', 0.11)])
cv_intercept 103.33731439645975 <class 'numpy.float64'>
COEF [0.48756847 0.19834493 0.61875

C:\projects\pkogo\env_ogo\lib\site-packages\sklearn\linear_model\_quantile.py:186: FutureWarning: The default solver will change from 'interior-point' to 'highs' in version 1.4. Set `solver='highs'` or to the desired solver to silence this warning.
  warnings.warn(
